In [27]:
import tensorflow as tf
import numpy as np
from libs.activations import lrelu

class AEModel(object):
    def __init__(self) :
        self.HIDDEN_STATE_SIZE = 128
        self.SCREEN_HEIGHT = 210
        self.SCREEN_WIDTH = 160
        self.n_filters = [1, 16, 32, 64]
        self.filter_sizes = 3
        self.input_shape = [None, 33600]
        self.keep_prob = 0.8

        self.build()


    def build(self):
        tf.reset_default_graph()
        self.x = tf.placeholder(tf.float32, self.input_shape, name='x')
        
        current_input = tf.reshape(self.x, [-1, self.SCREEN_HEIGHT, self.SCREEN_WIDTH, 1])

        
        encoder = []
        shapes = []
        with tf.variable_scope("encoder"):
          with tf.variable_scope("cnn_layers"):
            for layer_i, n_output in enumerate(self.n_filters[1:]):
                n_input = current_input.get_shape().as_list()[3]
                shapes.append(current_input.get_shape().as_list())
                w = tf.Variable(tf.random_normal([self.filter_sizes, self.filter_sizes, n_input, n_output], stddev=0.1))
                b = tf.Variable(tf.random_normal([n_output], stddev=0.1))
                encoder.append(w)
                conv = tf.sigmoid(tf.add(tf.nn.conv2d(current_input, w, strides=[1, 2, 2, 1], padding='SAME'), b))
                drop = tf.nn.dropout(conv, keep_prob = self.keep_prob)
                current_input = drop
            cnn_output_shape = current_input.get_shape().as_list()
            cnn_output_size = cnn_output_shape[1] * cnn_output_shape[2] * cnn_output_shape[3]
          with tf.variable_scope("dense_layers"):
            flatten = tf.contrib.layers.flatten(inputs = current_input)
            dense1 = tf.contrib.layers.fully_connected(flatten, num_outputs=256, activation_fn=tf.sigmoid)
            dense_drop1 = tf.contrib.layers.dropout(inputs = dense1, keep_prob = self.keep_prob)
                
            dense2 = tf.contrib.layers.fully_connected(inputs = dense_drop1, num_outputs=self.HIDDEN_STATE_SIZE, activation_fn=tf.sigmoid)
            dense_drop2 = tf.contrib.layers.dropout(inputs = dense2, keep_prob = self.keep_prob)
        
        with tf.variable_scope("hidden"):
          dense_drop2 = dense_drop2 * 255
          self.hidden = tf.cast(dense_drop2, tf.int32)

        encoder.reverse()
        shapes.reverse()
        with tf.variable_scope("decoder"):
          with tf.variable_scope("dense_layers"):
            dense3 = tf.contrib.layers.fully_connected(inputs = dense2, num_outputs = 256, activation_fn=tf.sigmoid)
            dense_drop3 = tf.contrib.layers.dropout(inputs = dense3, keep_prob = self.keep_prob)
                
            dense4 = tf.contrib.layers.fully_connected(inputs = dense_drop3, num_outputs = cnn_output_size, activation_fn=tf.sigmoid)
            dense_drop4 = tf.contrib.layers.dropout(inputs = dense4, keep_prob = self.keep_prob)

            reshape = tf.reshape(dense_drop4, [-1, cnn_output_shape[1], cnn_output_shape[2], cnn_output_shape[3]])
          with tf.variable_scope("cnn_transpose_layers"):
            for layer_i, shape in enumerate(shapes):
                w = encoder[layer_i]
                b = tf.Variable(tf.random_normal([w.get_shape().as_list()[2]], stddev=0.1))
                conv_transpose = tf.sigmoid(tf.add(tf.nn.conv2d_transpose(current_input, w,
                        tf.stack([tf.shape(self.x)[0], shape[1], shape[2], shape[3]]),
                        strides=[1, 2, 2, 1],
                        padding='SAME'), b))
                drop = tf.nn.dropout(conv_transpose, self.keep_prob)
                current_input = drop
        current_input = current_input * 255
        self.predict = tf.reshape(current_input, [-1, self.input_shape[1]], name = "predict")
        
        with tf.variable_scope("cost"):
          #tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.output, labels=self.Y))
          self.cost = tf.reduce_sum(tf.square(self.predict - self.x), name="cost")
          tf.summary.scalar("cost", self.cost)

        with tf.variable_scope("optimize"):
          learning_rate = 0.01
          self.optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer").minimize(self.cost)

        self.merged = tf.summary.merge_all()

In [ ]:
batch_size = 128

n_train_screens = 2432
screen_height = 210
screen_width = 160

dev_screens = np.zeros((batch_size, screen_height*screen_width))
n_dev_screens = batch_size

screen_dir = "./screens/alien/"
current_pos = 1
def nextBatch(dir):
    batch_screen = np.zeros((batch_size, screen_height*screen_width))
    
    global current_pos
    for i in range(0, batch_size):
        path = dir + str(current_pos + i) + ".matrix"
        with open(path, "r") as f:
            pixels = f.read().split(' ')[:-1]
            pixels = list(map(int, pixels))
            batch_screen[i] = np.array(pixels)

    current_pos += batch_size
    if current_pos >= n_train_screens:
        current_pos = 1
    return batch_screen

def train():
    ae = AEModel()
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()  # max_to_keep=1
    writer = tf.summary.FileWriter("./AE_nn_log", sess.graph)

    print("training...")
    # Fit all training data
    n_epochs = 50
    for epoch_i in range(n_epochs):
        for batch_i in range(n_train_screens // batch_size):
            #print(batch_i)
            batch_xs = nextBatch(screen_dir)
            sess.run(ae.optimizer, feed_dict={ae.x: batch_xs})
        print(epoch_i, sess.run(ae.cost, feed_dict={ae.x: batch_xs}), sess.run(ae.cost, feed_dict={ae.x: dev_screens}))
        summary, cost = sess.run([ae.merged, ae.cost], feed_dict={ae.x: dev_screens})
        writer.add_summary(summary, epoch_i)
    saver.save(sess, './ckpt/model')

    writer.close()
    sess.close()


for i in range(0, batch_size):
    path = screen_dir + str(n_train_screens + i) + ".matrix"
    with open(path, "r") as f:
        pixels = f.read().split(' ')[:-1]
        pixels = list(map(int, pixels))
        dev_screens[i] = np.array(pixels)
train()


training...
0 34412450000.0 34294512000.0
1 33066142000.0 32981033000.0
2 31709182000.0 31623640000.0
3 27060978000.0 26968603000.0
4 17721467000.0 17594565000.0
5 13353987000.0 13249427000.0
6 11550730000.0 11462216000.0
7 10697461000.0 10577870000.0
8 10238936000.0 10179413000.0
9 9958548000.0 9824651000.0


In [26]:
import matplotlib.pyplot as plt

ae = AEModel()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, "./ckpt/model")

def predict(screen_values):
    screen = np.array(screen_values)
    screen = np.reshape(screen, [1, 33600])
    hidden_state = sess.run(ae.z, feed_dict={ae.x: screen})
    hidden_state = np.reshape(hidden_state, [34560])
    return [v for v in hidden_state]


n_train_screens = 2432
n_dev_screens = 2
screen_height = 210
screen_width = 160
dev_screens = np.zeros((n_dev_screens, screen_height*screen_width))
screen_dir = "./screens/alien/"

def loadData(dir):
    for i in range(n_dev_screens):
        path = dir + str(n_train_screens+i+1) + ".matrix"
        with open(path, "r") as f:
            pixels = f.read().split(' ')[:-1]
            pixels = list(map(int, pixels))
            dev_screens[i] = np.array(pixels)
loadData(screen_dir)

n_examples = 2
recon = sess.run(ae.predict, feed_dict={ae.x: dev_screens})


fig, axs = plt.subplots(n_examples, 2, figsize=(210, 160), squeeze=False)
for example_i in range(n_examples):
    axs[example_i][0].imshow(
        np.reshape(dev_screens[example_i, :], (210, 160)))
    axs[example_i][1].imshow(
        np.reshape(recon[example_i, :], (210, 160)))
fig.show()
plt.draw()

sess.close()

INFO:tensorflow:Restoring parameters from ./ckpt/model


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/figure.py:459: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
